# Similarity Search With Postgres and Ollama

The notebook demonstrates how to perform similarity search using Postgres and an Ollama embedding model.

**0. Prerequsites**

* Install python and pip
* Start a Postgres container with pgvector and upload the movies dataset following the instructions from [chapter8.md](../chapter8.md)
* Start an Ollama container and download required embedding and large language models follow the [ai_samples/README.md](README.md) instructions.

**1. Install Required Modules**

In [ ]:
pip install -q psycopg2-binary==2.9.9 langchain-ollama==0.2.3

**2. Initialize Ollama Embedding Model**

In [4]:
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model="mxbai-embed-large:335m")

**3. Connect to Postgres**

In [9]:
import psycopg2

db_params = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "postgres",
    "password": "password"
}

conn = psycopg2.connect(**db_params)

**4. Perform Similarity Search**

In [ ]:
# Enter the search phrase
phrase = "A team of superheroes come together to save the world"

cursor = conn.cursor()

# Generating embedding 
embedding = embedding_model.embed_query(phrase)

# Execute the SQL query to find similar movies
query = """
SELECT id, name, description, 1 - (movie_embedding <=> %s::vector) as similarity
FROM omdb.movies
WHERE 1 - (movie_embedding <=> %s::vector) > 0.3
ORDER BY similarity DESC
LIMIT 3
"""

cursor.execute(query, (embedding, embedding))
result = cursor.fetchall()

# Print the results
print(f'Search result for phrase "{phrase}"')
for row in result:
    print(row)

# Close the cursor and connection
cursor.close()

**5. Close Database Connection**

In [15]:
conn.close()